# Short Story Generator

## Constants

In [ ]:
my_path = ""

## Input options

In [ ]:
random = "Once upon a time"
constructed = "This is Bob. Bob is a cat."

## Model Encapsulators

In [ ]:
%run Mods.ipynb

## Post-processing

In [ ]:
def post_proc(raw_story):
    return raw_story

## Model Prediction Pipeline

In [ ]:
def compose_story(model, training_set, seed_sentence):
    raw_output = ""
    
    if model == 'ngram_2':
            if training_set == 'ts1':
                raw_output = n_gram_2_t1(seed_sentence)
            elif training_set == 'ts2':
                raw_output = n_gram_2_t2(seed_sentence)
            else: 
                print('Invalid training set for selected model')
    elif model == 'ngram_3':
            if training_set == 'ts2':
                raw_output = n_gram_3_t3(seed_sentence)
            elif training_set == 'ts2':
                raw_output = n_gram_3_t3(seed_sentence)
            else: 
                print('Invalid training set for selected model')   
    else:
        print("Invalid model")
    
    return post_proc(raw_output)

In [ ]:
compose_story(model = 'ngram_3', training_set = 'ts2', seed_sentence = 'Hi')

## Evaluation Grid Generator

In [ ]:
from IPython.display import clear_output
import pandas as pd

In [ ]:
Seed_sentences = [
    random,
    constructed
]

Models = [
    'ngram_2',
    'ngram_3'
]

Training_sets = ['ts1', 'ts2', 'ts3', 'ts4']

In [ ]:
Stories = pd.DataFrame(columns=['Model',
                    'Training_set',
                    'Seed_sentence',
                    'Story'])

In [ ]:
for Model in Models:
    for Training_set in Training_sets:
        for Seed_sentence in Seed_sentences:
            print("Composing Story using model =", Model, ", training_set = ",Training_set, " and, seed_sentence = ", Seed_sentence)
            story = compose_story(model=Model, training_set=Training_set, seed_sentence=Seed_sentence)
            if story != "":
                entry = {
                    'Model' : Model,
                    'Training_set': Training_set,
                    'Seed_sentence' : Seed_sentence,
                    'Story' : story
                }
                Stories = Stories.append(entry, True)

In [ ]:
Stories

In [ ]:
Stories = Stories.sample(frac=1).reset_index(drop=True)
Stories

In [ ]:
Stories.to_csv(my_path + "RawEvaluationGrid.csv")

## Rating Session

Functions

In [ ]:
def rating(prompt, min_rating, max_rating):
    answer = input(prompt)
    answer = int(answer)
    return answer

In [ ]:
def rating_loop(EvaluationGrid, rater_name):
    H_line = "---------------------------------------------------------------------------"
    Header = (H_line + "\n" + "Manual evaluation in progress.\nRater: " + rater_name + "\n" + H_line)
    max_ind = len(EvaluationGrid)
    
    for ind in EvaluationGrid.index:
        while True:
            clear_output()
            print(Header)
            print("Story number %d of %d to be evaluated.\nStory:"%(ind+1, max_ind))
            print(EvaluationGrid.loc[ind, 'Story'])

            creativity = rating('Creativity: ', 1, 5)
            correctness = rating('Correctness: ', 1, 5)
            child_friendliness = rating('Child-friendliness: ', 1, 5)

            satisfied = input("Are you satisfied with these ratings (type 'y' if yes)?")
            if satisfied == 'y':
                EvaluationGrid.loc[ind, 'creativity'] = creativity
                EvaluationGrid.loc[ind, 'correctness'] = correctness
                EvaluationGrid.loc[ind, 'child_friendliness'] = child_friendliness
                break

Code to edit:

In [ ]:
EvaluationGrid = pd.read_csv(my_path + "RawEvaluationGrid.csv", index_col=0)
rater_name = "Rater"

Code to run to rate all stories from the begining:

In [ ]:
if input("Type 'y' if you are sure you want to rate all stories from the begining") == "y":
    rating_loop(EvaluationGrid, rater_name)

In [ ]:
Stories

In [ ]:
EvaluationGrid.to_csv(my_path + "EvaluationGrid-" + rater_name + ".csv")